In [1]:
from pathlib import Path
import sys

BASE_PATH = Path('../')
PATH_TO_DATA = BASE_PATH/'data'
PATH_TO_MODELS = BASE_PATH/'checkpoints'

PATH_TO_DATA.mkdir(exist_ok=True, parents=True)
PATH_TO_MODELS.mkdir(exist_ok=True, parents=True)

sys.path.append(str(BASE_PATH/'..'))

#### <b>Load Libraries</b>

In [2]:
from tqdm import tqdm
import time
import random
from pathlib import Path

from PIL import Image

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import models
from torchvision import transforms

from torch.utils.data import Dataset, DataLoader, ConcatDataset

from utils import progress_bar, load_model

stty: stdin isn't a terminal


#### <b>Notebook Constants</b>

In [3]:
RANDOM_STATE = 42
BATCH_SIZE = 64
NUM_WORKERS = 0
DEVICE = 'mps'

In [4]:
def set_random_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

#### <b>Define Custom Dataset</b>

In [5]:
class CustomDataset(Dataset):
    def __init__(self, manifest, image_directory, train_mode):

        if train_mode:
            self.transform = transforms.Compose([
                transforms.Resize(128),
                transforms.RandomHorizontalFlip(),
                transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                transforms.ToTensor()
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize(128),
                transforms.ToTensor()
            ])

        image_directory = Path(image_directory)
        age_class_to_label = {
            "a": 0, "b": 1, "c": 2, "d": 3, "e": 4, "f": 5, "g": 6, "h": 7
        }
        
        self.filenames = [image_directory/x for x in manifest['image_path']]
        self.labels = [age_class_to_label[x] for x in manifest['age_class']]

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img = self.transform(Image.open(self.filenames[idx]))
        label = self.labels[idx]

        return img, label

#### <b>Evalution utils</b>

In [6]:
def downstream_task_performance(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            outputs = model(inputs)

            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(dataloader), f'Acc: {100*correct/total:.2f}%%%')  
    
    return 100.*correct/total

In [13]:
from MIA.score import get_logits, lira_offline

def lira(target_net, forget_manifest_filename, retain_manifest_filename):

    num_aug = 1
    fix_variance = False

    train_mode = True if num_aug > 1 else False
    forget_manifest = pd.read_csv(PATH_TO_DATA/forget_manifest_filename)
    retain_manifest = pd.read_csv(PATH_TO_DATA/retain_manifest_filename)
    forget_dataset = CustomDataset(forget_manifest, PATH_TO_DATA/'images', train_mode=train_mode)
    retain_dataset = CustomDataset(retain_manifest, PATH_TO_DATA/'images', train_mode=train_mode)
    dataset = ConcatDataset([forget_dataset, retain_dataset])
    dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

    shadow_nets = [load_model(state_dict).to(DEVICE) for state_dict in (PATH_TO_MODELS/'shadow').glob('*/*')]

    shadow_logits = []
    target_logits = []
    dataset_labels = []

    for _ in range(num_aug):
        shadow_aug_logits = [[] for _ in range(len(shadow_nets))]
        target_aug_logits = []
        with torch.no_grad():
            for batch_idx, (inputs, targets) in tqdm(enumerate(dataloader), total=len(dataloader)):
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

                # 1. Store shadow logits
                for i, net in enumerate(shadow_nets):
                    logits = net(inputs).cpu().tolist()
                    shadow_aug_logits[i].append(logits)
                # 2. Store target logits
                logits = target_net(inputs).cpu().tolist()
                target_aug_logits.append(logits)
            
                dataset_labels.append(targets.cpu().tolist())

        shadow_aug_logits = np.stack([np.concatenate(logits) for logits in shadow_aug_logits])  # [n_shadow, n_examples, n_classes]
        target_aug_logits = np.concatenate(target_aug_logits)  # [n_examples, n_classes]
        dataset_labels = np.concatenate(dataset_labels)[:, None]  # [n_examples, 1]

        # Extract numerically stable logits
        shadow_aug_logits = get_logits(shadow_aug_logits, np.repeat(dataset_labels[None], len(shadow_nets), 0))
        target_aug_logits = get_logits(target_aug_logits, dataset_labels)

        shadow_logits.append(shadow_aug_logits)
        target_logits.append(target_aug_logits)

        shadow_logits = np.stack(shadow_logits, axis=-1)
        shadow_logits = np.swapaxes(shadow_logits, 0, 1)  # [n_examples, n_shadow, n_aug]
        target_logits = np.stack(target_logits, axis=-1)  # [n_examples, n_aug]
        labels = np.array([0] * len(forget_dataset) + [1] * len(retain_dataset)) # [n_examples]

        fnr, tnr, low = lira_offline(target_logits, shadow_logits, labels, fix_variance=fix_variance)

    return fnr, tnr, low

In [14]:
def evaluate_unlearning(state_dict_path):
    model = load_model(state_dict_path)
    model.to(DEVICE)

    train_manifest = pd.read_csv(PATH_TO_DATA/'train_manifest.csv')
    train_dataset = CustomDataset(train_manifest, PATH_TO_DATA/'images', train_mode=False)
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)

    test_manifest = pd.read_csv(PATH_TO_DATA/'test_manifest.csv')
    test_dataset = CustomDataset(test_manifest, PATH_TO_DATA/'images', train_mode=False)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)

    train_acc = downstream_task_performance(model, train_dataloader)
    test_acc = downstream_task_performance(model, test_dataloader)
    mia = lira(model, 'forget_manifest.csv', 'retain_manifest.csv')

    return train_acc, test_acc, mia

#### <b>Evaluate origin model</b>

In [15]:
evaluate_unlearning(PATH_TO_MODELS/'origin'/'state_dict.pth')

 [=======================================>]  Step: 26ms | Tot: 4s595ms | Acc: {100*correct/total:.2f}% 85/85 
 [======================================>.]  Step: 10ms | Tot: 1s237ms | Acc: {100*correct/total:.2f}% 25/25 


 54%|█████▍    | 46/85 [02:12<01:52,  2.87s/it]


KeyboardInterrupt: 